## Imports

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import datetime
import string
import json
import nltk
from nltk import pos_tag
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import warnings
warnings.filterwarnings('ignore')

# 1. Crawler

In [2]:
URL = "https://pureportal.coventry.ac.uk/en/organisations/coventry-university/persons/"
profile_url = "https://pureportal.coventry.ac.uk/en/persons/"

In [3]:
def get_maximum_page():
    
    first = requests.get(URL)
    soup = BeautifulSoup(first.text, 'html.parser')
    final_page = soup.select('#main-content > div > section > nav > ul > li:nth-child(12) > a')[0]['href']
    fp = final_page.split('=')[-1]
    return int(fp)
    
mx = get_maximum_page()    

In [4]:
def check_department(researcher):
    
    l1 = researcher.find('div', class_='rendering_person_short')
      
    for span in l1.find_all('span'):
        if span.text == str('School of Economics, Finance and Accounting'):
            name = researcher.find('h3', class_='title').find('span').text
            return name
        else:
            pass

def create_csv():
    database = pd.DataFrame(columns=['Title', 'Author', 'Published', 'Link'])
    database.to_csv('data.csv')
    
def update_csv(database):
    current_data = pd.read_csv(database, index_col="Unnamed: 0")
    return current_data        

def enter_each_researchers_publication(researcher, url, df):
    
    new_url = url + str(researcher).replace(' ','-').lower() + '/publications/'
    page = requests.get(new_url)
    soup = BeautifulSoup(page.content, "html.parser")
    results = soup.find(id="main-content")
    papers = results.find_all("li", class_="list-result-item")
    
    
    for paper in papers:
        title = paper.find('h3', class_='title').find('span')
        author = paper.find('a', class_='link person').find('span')
        date = paper.find('span', class_="date")
        link = paper.find('h3', class_='title').find('a', href=True)['href']
        
        opening = pd.read_csv('data.csv', index_col="Unnamed: 0")
        opening = opening.append({'Title': title.text, 
                                  'Author': author.text, 
                                  'Published': date.text,
                                  'Link': link}, ignore_index=True)
        opening.to_csv('data.csv')     

In [5]:
# Scraping function
def scrap(mx):
    df = update_csv('data.csv')
    i=0
    while True:
    
        if i > 17:
            break
            
        if i>0:
            url = URL + '?page=' + str(i)
        else:
            url = URL
    
        i = i+1
        # scraping starts here
        page = requests.get(url)
        soup = BeautifulSoup(page.content, "html.parser")
        results = soup.find(id="main-content")
        researchers = results.find_all("li", class_="grid-result-item")

        for researcher in researchers:
            # Check if researcher has any papers
            check = researcher.find('div', class_='stacked-trend-widget')
            if check:
                name = check_department(researcher)
                if name is None:
                    pass
                else:
                    enter_each_researchers_publication(name, profile_url, df)

In [6]:
create_csv()

%time scrap(mx) 

CPU times: user 4.77 s, sys: 146 ms, total: 4.92 s
Wall time: 1min 49s


In [7]:
df = pd.read_csv('data.csv').rename(columns={'Unnamed: 0':'SN'})
df
print(f'{df.shape[0]} Scraped Records')

86 Scraped Records


# 2. Indexing

In [8]:
scraped_df = pd.read_csv('data.csv').rename(columns={'Unnamed: 0':'SN'}).reset_index(drop=True)
scraped_df.head()

,SN,Title,Author,Published,Link
0,0,Stock Liquidity and SMEs’ Likelihood of Bankru...,"Abd Karim, M. N.",Dec 2017,https://pureportal.coventry.ac.uk/en/publicati...
1,1,"Personalisation of power, neoliberalism and th...","Mohamad, M. N.",1 Sep 2015,https://pureportal.coventry.ac.uk/en/publicati...
2,2,Stock Price and Volume Effects Associated with...,"Abd Karim, M. N.",11 Feb 2014,https://pureportal.coventry.ac.uk/en/publicati...
3,3,Competing Institutional Logics and Power Dynam...,"Abras, A.",10 May 2022,https://pureportal.coventry.ac.uk/en/publicati...
4,4,The rise and fall of institutional entrepreneu...,"Abras, A.",10 Apr 2022,https://pureportal.coventry.ac.uk/en/publicati...


In [9]:
df.head(7)

,SN,Title,Author,Published,Link
0,0,Stock Liquidity and SMEs’ Likelihood of Bankru...,"Abd Karim, M. N.",Dec 2017,https://pureportal.coventry.ac.uk/en/publicati...
1,1,"Personalisation of power, neoliberalism and th...","Mohamad, M. N.",1 Sep 2015,https://pureportal.coventry.ac.uk/en/publicati...
2,2,Stock Price and Volume Effects Associated with...,"Abd Karim, M. N.",11 Feb 2014,https://pureportal.coventry.ac.uk/en/publicati...
3,3,Competing Institutional Logics and Power Dynam...,"Abras, A.",10 May 2022,https://pureportal.coventry.ac.uk/en/publicati...
4,4,The rise and fall of institutional entrepreneu...,"Abras, A.",10 Apr 2022,https://pureportal.coventry.ac.uk/en/publicati...
5,5,Institutional entrepreneurship in Islamic fina...,"Abras, A.",9 Jan 2020,https://pureportal.coventry.ac.uk/en/publicati...
6,6,Competing institutional logics and institution...,"Abras, A.",2019,https://pureportal.coventry.ac.uk/en/publicati...


In [10]:
single_row = scraped_df.loc[1,:].copy()
single_row

SN                                                           1
Title        Personalisation of power, neoliberalism and th...
Author                                          Mohamad, M. N.
Published                                           1 Sep 2015
Link         https://pureportal.coventry.ac.uk/en/publicati...
Name: 1, dtype: object

## 2.1 Preprocess Text

In [11]:
# Stopwords Removal
# Lowercasing the data
# Removing punctuation
sw = stopwords.words("english")
lemmatizer = WordNetLemmatizer()


def tp1(txt):
    txt = txt.lower()   
    txt = txt.translate(str.maketrans('',
                                      '',
                                      string.punctuation))   
    txt = lematize(txt)
    return txt


def fwpt(word):
    tag = pos_tag([word])[0][1][0].upper()
    hash_tag = {"V": wordnet.VERB, "R": wordnet.ADV,"N": wordnet.NOUN,"J": wordnet.ADJ}         
    return hash_tag.get(tag, wordnet.NOUN)

def lematize(text):
        tkns = nltk.word_tokenize(text)
        ax = ""
        for each in tkns:
            if each not in sw:
                ax += lemmatizer.lemmatize(each, fwpt(each)) + " "
        return ax


### Without pre-processing

In [12]:
df['Title'].iloc[5]

'Institutional entrepreneurship in Islamic financial reporting standardisation projects'

### With pre-processing

In [13]:
scraped_df['Title'].iloc[5]

'Institutional entrepreneurship in Islamic financial reporting standardisation projects'

### 2.1.1 Preprocessing complete dataframe

In [14]:
processed_df = scraped_df.copy()

def preprocess_df(df):
    df.Title = df.Title.apply(tp1)
    df.Author = df.Author.str.lower()
    df = df.drop(columns=['Author','Published'], axis=1)
    return df
    
preprocess_df(processed_df)
processed_df.head()

,SN,Title,Author,Published,Link
0,0,stock liquidity smes ’ likelihood bankruptcy e...,"abd karim, m. n.",Dec 2017,https://pureportal.coventry.ac.uk/en/publicati...
1,1,personalisation power neoliberalism production...,"mohamad, m. n.",1 Sep 2015,https://pureportal.coventry.ac.uk/en/publicati...
2,2,stock price volume effect associate change com...,"abd karim, m. n.",11 Feb 2014,https://pureportal.coventry.ac.uk/en/publicati...
3,3,compete institutional logic power dynamic isla...,"abras, a.",10 May 2022,https://pureportal.coventry.ac.uk/en/publicati...
4,4,rise fall institutional entrepreneurship islam...,"abras, a.",10 Apr 2022,https://pureportal.coventry.ac.uk/en/publicati...


## 2.2 Index Construction

In [15]:
single = processed_df.loc[0,:].copy()
print(single)
indexing_trial = {}

words = single.Title.split()
SN = single.SN
word = words[0]
example = {word: [SN]}

print('Sample index')
print(example)

SN                                                           0
Title        stock liquidity smes ’ likelihood bankruptcy e...
Author                                        abd karim, m. n.
Published                                             Dec 2017
Link         https://pureportal.coventry.ac.uk/en/publicati...
Name: 0, dtype: object
Sample index
{'stock': [0]}


In [16]:
## Indexer Function
def apply_index(inputs, index):
    words = inputs.Title.split()
    SN = int(inputs.SN)
    for word in words:
        if word in index.keys():
            if SN not in index[word]:
                index[word].append(SN)
        else:
            index[word] = [SN]
    return index

indx = apply_index(inputs=single, index= {})

In [17]:
def full_index(df, index):
    for x in range(len(df)):
        inpt = df.loc[x,:]
        ind = apply_index(inputs=inpt, index=index)
    return ind

def construct_index(df, index):
    queue = preprocess_df(df)
    ind = full_index(df=queue, index=index)
    return ind

indexed = full_index(processed_df, 
                     index = {})


indexes = construct_index(df=scraped_df, 
                          index = {})

In [18]:
with open('index.json', 'w') as new_f:
    json.dump(indexes, new_f, sort_keys=True, indent=4)
    
with open('index.json', 'r') as file:
    data = json.load(file)

def index_2(df, x_path):
    if len(df) > 0:
        with open(x_path, 'r') as file:
            prior_index = json.load(file)
        new_index = construct_index(df = df, index = prior_index)
        with open(x_path, 'w') as new_f:
            json.dump(new_index, new_f, sort_keys=True, indent=4)

In [19]:
len(data)

376

## 3.  Query Processor

In [20]:
def demonstrate_query_processing():
    sample = input('Enter Search Terms: ')
    processed_query = tp1(sample)
    print(f'Processed Search Query: {processed_query}')
    return processed_query

### 3.1.  Split Query into individual terms

In [21]:
def split_query(terms):
    each = tp1(terms)
    return each.split()

dqp = demonstrate_query_processing()
dqp
print(f'Split Query: {split_query(dqp)}')

Enter Search Terms: Finance and Accounting
Processed Search Query: finance accounting 
Split Query: ['finance', 'accounting']


### 3.2.  Boolean Functionalities

In [22]:
def union(lists):
    union = list(set.union(*map(set, lists)))
    union.sort()
    return union

def intersection(lists):
    intersect = list(set.intersection(*map(set, lists)))
    intersect.sort()
    return intersect

### 3.3. Search Engine Function

In [23]:
def vertical_search_engine(df, query, index=indexes):
    query_split = split_query(query)
    retrieved = []
    for word in query_split:
        if word in index.keys():
            retrieved.append(index[word])
            
            
    # Ranked Retrieval
    if len(retrieved)>0:
        high_rank_result = intersection(retrieved)
        low_rank_result = union(retrieved) 
        c = [x for x in low_rank_result if x not in high_rank_result]      
        high_rank_result.extend(c)
        result = high_rank_result
        
        final_output = df[df.SN.isin(result)].reset_index(drop=True)
    
        # Return result in order of Intersection ----> Union
        dummy = pd.Series(result, name = 'SN').to_frame()
        result = pd.merge(dummy, final_output, on='SN', how = 'left')
        
    else:
        result = 'No result found'
    
    return result

In [24]:
def test_search_engine():
    xtest = scraped_df.copy()
    query = input("Enter your search query: ")
    return vertical_search_engine(xtest, query, indexed)
    
test_search_engine()

Enter your search query: School of Finance and Accounting


,SN,Title,Author,Published,Link
0,22,financial accounting principle islamic finance,"alamad, s.",2019,https://pureportal.coventry.ac.uk/en/publicati...
1,24,financial innovation engineering islamic finance,"alamad, s.",2017,https://pureportal.coventry.ac.uk/en/publicati...
2,25,islamic finance ethic perception western consu...,"alamad, s.",13 Mar 2017,https://pureportal.coventry.ac.uk/en/publicati...
3,26,risk management methodology empirical macropru...,"alamad, s.",26 May 2016,https://pureportal.coventry.ac.uk/en/publicati...
4,35,fair value accounting value relevance equity b...,"alhaj ismail, a.",Jun 2020,https://pureportal.coventry.ac.uk/en/publicati...
5,36,accounting treatment sharebased payment impact...,"alhaj ismail, a.",Dec 2019,https://pureportal.coventry.ac.uk/en/publicati...
6,43,smart city dynamic multilevel management accou...,"alsaid, l.",3 Jan 2022,https://pureportal.coventry.ac.uk/en/publicati...
7,46,accounting smart city new evidence governmenta...,"alsaid, l.",9 Apr 2020,https://pureportal.coventry.ac.uk/en/publicati...
8,47,accounting structural reform case study egypti...,"alsaid, l.",Jan 2018,https://pureportal.coventry.ac.uk/en/publicati...
9,83,antecedent consequence worldwide diffusion acc...,"elmghaamez, i.",11 Mar 2019,https://pureportal.coventry.ac.uk/en/publicati...


In [25]:
def final_engine(results):
    if type(results) != 'list':
        return results
    else:
        for i in range(len(results)):
            printout = results.loc[i, :]
            print(printout['Title'])
            print(printout['Author'])
            print(printout['Published'])
            print(printout['Link'])
            print('')

In [26]:
scraped_df['Title'].iloc[24]

'financial innovation engineering islamic finance '

In [27]:
final_engine(test_search_engine())

Enter your search query: School of Finance and Accounting


,SN,Title,Author,Published,Link
0,22,financial accounting principle islamic finance,"alamad, s.",2019,https://pureportal.coventry.ac.uk/en/publicati...
1,24,financial innovation engineering islamic finance,"alamad, s.",2017,https://pureportal.coventry.ac.uk/en/publicati...
2,25,islamic finance ethic perception western consu...,"alamad, s.",13 Mar 2017,https://pureportal.coventry.ac.uk/en/publicati...
3,26,risk management methodology empirical macropru...,"alamad, s.",26 May 2016,https://pureportal.coventry.ac.uk/en/publicati...
4,35,fair value accounting value relevance equity b...,"alhaj ismail, a.",Jun 2020,https://pureportal.coventry.ac.uk/en/publicati...
5,36,accounting treatment sharebased payment impact...,"alhaj ismail, a.",Dec 2019,https://pureportal.coventry.ac.uk/en/publicati...
6,43,smart city dynamic multilevel management accou...,"alsaid, l.",3 Jan 2022,https://pureportal.coventry.ac.uk/en/publicati...
7,46,accounting smart city new evidence governmenta...,"alsaid, l.",9 Apr 2020,https://pureportal.coventry.ac.uk/en/publicati...
8,47,accounting structural reform case study egypti...,"alsaid, l.",Jan 2018,https://pureportal.coventry.ac.uk/en/publicati...
9,83,antecedent consequence worldwide diffusion acc...,"elmghaamez, i.",11 Mar 2019,https://pureportal.coventry.ac.uk/en/publicati...


## 4. Schedule Crawler for every week

In [28]:
days = 0
interval = 7
while days <= 1:
    scrap(mx)
    print(f"Crawled at {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print(f'Next crawl scheduled after {interval} days')
    time.sleep(interval)
    days = days + 1

Crawled at 2022-07-29 17:38:22
Next crawl scheduled after 7 days
Crawled at 2022-07-29 17:40:10
Next crawl scheduled after 7 days


# 5. GUI

In [29]:
from tkinter import *
from PIL import Image, ImageTk 
from tkinter import scrolledtext
from pandastable import Table, TableModel
from contextlib import suppress
import warnings
warnings.filterwarnings('ignore')

image1 = Image.open('coventry-logo-1.png')
resized_image1 = image1.resize((5,2), Image.ANTIALIAS)

def new_gui(pic):
    window = Tk()
    window.title("Coventry University Search Engine")
    window.geometry('200x150')
    
    lbl = Label(window, text="Publication Search Engine", font=("Arial Bold", 30), padx=5, pady=5)
    lbl.grid(column=1, row=0)
    lbl2 = Label(window, text="Enter your search query: ", font=("Arial", 15), padx=5, pady=5)
    lbl2.grid(column=0, row=1)
    
    img = ImageTk.PhotoImage(pic)
    
    lbl3 = Label(image=img)
    lbl3.image = img
    lbl3.grid(column=1, row=3, padx=5, pady=5)

    query = Entry(window,width=40)
    query.grid(column=1, row=1,  padx=5, pady=5)
    
    results = Canvas(window, height=15, width=150)
    results.grid(column=1, row=2, padx=5, pady=5)
    
    # Entry
    def getInputBoxValue():
        userInput = query.get()
        return userInput

    
    # Button
    def clicked():
        search()
        #pass
        
    def no_result():
        messagebox.showwarning("Warning", "No results found. Please try different search terms")
        
    
    def search():
        xtest = scraped_df.copy()
        q = query.get()
        f = Frame(window)
        df = vertical_search_engine(xtest, q, indexed)
        if type(df) == str:
            no_result()
        else:
            pt = Table(results)
            try:
                table = pt = Table(results, dataframe=df)
                pt.show()
            except AttributeError:
                pass
            
    def close_window():
        if messagebox.askokcancel("Quit", "Quit Program?"):
            window.destroy()
        
    
    btn = Button(window, text="Search", command=clicked)
    btn.grid(column=2, row=1)
    
    

    window.protocol("Delete_Window", close_window)       
    window.mainloop()

In [30]:
new_gui(image1)